<a href="https://colab.research.google.com/github/Hjhirp/RNN-Language-Model-Implementation/blob/main/Notebook/RNN_Language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import numpy
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

data = """ Jack and Jill went up the hill .\n To fetch a pail of water .\n Jack fell down and broke his crown .\n And Jill came tumbling after ."""
print("Data:", data, type(data))

data_splitted=data.split('\n') #returns a list of strings
print("Data_Splitted:", data_splitted, type(data_splitted))

tokenizer=Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\]^_`{|}~')

tokenizer.fit_on_texts(data_splitted) #learns a vocabulary
print("Word Indices:", tokenizer.word_index) #tokenizer.word_index is a dictionary

vocab_size = len(tokenizer.word_index) + 1
print("Vocab Size:", vocab_size)

sequences=tokenizer.texts_to_sequences(data_splitted) #list of list
l=len(sequences)
print("Sequences:",sequences, type(sequences), l)

X=list()
y=list()

for i in range(len(sequences)):
  X.insert(i,sequences[i][:-1])
  y.insert(i,sequences[i])

print("X=", X, "y=",y, type(X), type(y))

maxlen = max([len(sequence) for sequence in X])
print("Maxlen:",maxlen)

#X=array(X)
#y=array(y)
#X=numpy.reshape(X,newshape=(l,-1))
#y=numpy.reshape(y,newshape=(l,-1))

X=pad_sequences(X,maxlen=maxlen+1,padding='pre') # +1 to have 0 as the first input
print("X:",X, type(X), X.shape)
y=pad_sequences(y,maxlen=maxlen+1,padding='pre')
print("y:",y, type(y), y.shape)

#z=numpy.
#for i in range(l):
#  z[i]=to_categorical(y[i],num_classes=vocab_size)

#print("X=", X, "y=",y)
y=to_categorical(y,num_classes=vocab_size)

print("X=", X, "y=",y, type(X), type(y), X.shape, y.shape)
model=Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10))
print(model.output_shape)
model.add(SimpleRNN(units=50, return_sequences=True))
print(model.output_shape)
model.add(Dense(units=vocab_size,activation='softmax'))
model.summary()
print("Input Shape of all Layers:",model.layers[0].input_shape,model.layers[1].input_shape,model.layers[2].input_shape)
print("Input Dim:",model.layers[0].input_dim)

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

model.fit(X, y, epochs=200, verbose=0)

'''def sample_seq_wo_seed(model, tokenizer, n_words, vocab_size): #only the first word is sampled
  encoded=list()
  in_text = ''
  # generate a fixed number of words = n_words
  for i in range(n_words):
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0] # for words not in the vocab it returns []
    # pre-pad sequences to a fixed length
    #encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    encoded.insert(0,0)
    encoded=array(encoded)
    encoded=numpy.reshape(encoded,newshape=(1,-1))
    print("i:", i, "Encoded:",encoded, encoded.shape)
    # predict probability and sample a word from vocab
    if i == 0:
      prob = model.predict_proba(encoded, verbose=0)
      print("i=", i, "Prob:", prob, type(prob), prob.shape)
      yhat=0
      while yhat == 0:
        yhat=numpy.random.choice(range(vocab_size),p=prob.ravel())
      yhat=[yhat]
      yhat=array(yhat)
      yhat=numpy.reshape(yhat,newshape=(1,-1))
      print("i:",i,"If yhat:", yhat, yhat.shape)
    else:
      yhat=numpy.append(yhat,0)
      yhat=numpy.reshape(yhat,newshape=(1,-1))
      while yhat[0,i] == 0:
        yhat=model.predict_classes(encoded, verbose=0)
      print("i:", i, "Else yhat:", yhat, yhat.shape)


    # map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat[0,i]:
        out_word = word
        #print("index:", index, "out_word:", out_word)
        break

    # append to input
    in_text = in_text + out_word + ' '
  return in_text
'''
#print(tokenizer.word_index.items())

'''print("------------------Sample Sequence without Seed---------------------------------")
print(sample_seq_wo_seed(model, tokenizer, 8, vocab_size))
print("-------------------------------------------------------------------------------")
'''

def sample_all_seq_wo_seed(model, tokenizer, n_words, vocab_size): #all the words are sampled
  encoded=list()
  in_text = ''
  # generate a fixed number of words = n_words
  for i in range(n_words):
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0] # for words not in the vocab it returns []
    print("i:", i, "Encoded:",encoded)
    # pre-pad sequences to a fixed length
    #encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    encoded.insert(0,0)
    encoded=array(encoded)
    encoded=numpy.reshape(encoded,newshape=(1,-1))
    print("i:", i, "Encoded:",encoded, encoded.shape)
    # predict probability and sample a word from vocab
    if i == 0:
      prob = model.predict_proba(encoded)
      print("i=", i, "Prob:", prob, type(prob), prob.shape)
      yhat=0
      while yhat == 0:
        yhat=numpy.random.choice(range(vocab_size),p=prob.ravel())
      yhat=[yhat]
      yhat=array(yhat)
      yhat=numpy.reshape(yhat,newshape=(1,-1))
      print("i:", i, "If yhat:", yhat, yhat.shape)
    else:
      prob = model.predict_proba(encoded)
      print("i=", i, "Prob:", prob, type(prob), prob.shape)
      yhat=numpy.append(yhat,0)  #just creating space for the next yhat
      yhat=numpy.reshape(yhat,newshape=(1,-1))
      while yhat[0,i] == 0:
        yhat[0,i]=numpy.random.choice(range(vocab_size),p=prob[0,i].ravel())
      print("i:", i, "Else yhat:", yhat, yhat.shape)


    # map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat[0,i]:
        out_word = word
        #print("index:", index, "out_word:", out_word)
        break

    # append to input
    in_text = in_text + out_word + ' '
  return in_text

print("-----------------------sample all seq without seed-----------------------------")
print(sample_all_seq_wo_seed(model, tokenizer, 8, vocab_size))
print("-------------------------------------------------------------------------------")


def prob_of_input_sentence(model, tokenizer, sentence):
  print("Input Sentence:", sentence)
  encoded=tokenizer.texts_to_sequences([sentence])[0]
  print("encoded before insert:", encoded)
  encoded.insert(0,0)
  encoded=array(encoded)
  encoded=numpy.reshape(encoded,newshape=(1,-1))
  print("Encoded:", encoded, encoded.shape)
  prob=model.predict_proba(encoded, verbose=0)
  print("Prob:", prob, prob.shape)
  probability=1
  for i in range(prob.shape[1]-1):
    probability = probability * prob[0,i,encoded[0,i+1]]
  print("Probability of Sentence", "\"", sentence, "\"", "is:", probability)

print("-------------------Probability of Input Sentence------------------------------")
prob_of_input_sentence(model, tokenizer, "Jack and Jill Went")
prob_of_input_sentence(model, tokenizer, "and Jill Went up")
prob_of_input_sentence(model, tokenizer, "went up the hill")
prob_of_input_sentence(model, tokenizer, "jack and Jill Went up the hill .")
prob_of_input_sentence(model, tokenizer, "to fetch a pail")
prob_of_input_sentence(model, tokenizer, "fetch a pail")
prob_of_input_sentence(model, tokenizer, "to fetch a pail of water")
prob_of_input_sentence(model, tokenizer, "jack fell down and")
prob_of_input_sentence(model, tokenizer, "jack fell down and broke")
prob_of_input_sentence(model, tokenizer, "fell down and broke")
prob_of_input_sentence(model, tokenizer, "jack fell down and broke his crown")
prob_of_input_sentence(model, tokenizer, "and jill came tumbling")
prob_of_input_sentence(model, tokenizer, "jill came tumbling after")
prob_of_input_sentence(model, tokenizer, "and jill came tumbling after .")
print("-------------------------------------------------------------------------------")

Data:  Jack and Jill went up the hill .
 To fetch a pail of water .
 Jack fell down and broke his crown .
 And Jill came tumbling after . <class 'str'>
Data_Splitted: [' Jack and Jill went up the hill .', ' To fetch a pail of water .', ' Jack fell down and broke his crown .', ' And Jill came tumbling after .'] <class 'list'>
Word Indices: {'.': 1, 'and': 2, 'jack': 3, 'jill': 4, 'went': 5, 'up': 6, 'the': 7, 'hill': 8, 'to': 9, 'fetch': 10, 'a': 11, 'pail': 12, 'of': 13, 'water': 14, 'fell': 15, 'down': 16, 'broke': 17, 'his': 18, 'crown': 19, 'came': 20, 'tumbling': 21, 'after': 22}
Vocab Size: 23
Sequences: [[3, 2, 4, 5, 6, 7, 8, 1], [9, 10, 11, 12, 13, 14, 1], [3, 15, 16, 2, 17, 18, 19, 1], [2, 4, 20, 21, 22, 1]] <class 'list'> 4
X= [[3, 2, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14], [3, 15, 16, 2, 17, 18, 19], [2, 4, 20, 21, 22]] y= [[3, 2, 4, 5, 6, 7, 8, 1], [9, 10, 11, 12, 13, 14, 1], [3, 15, 16, 2, 17, 18, 19, 1], [2, 4, 20, 21, 22, 1]] <class 'list'> <class 'list'>
Maxlen: 7
X: [[

AttributeError: ignored